# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [333]:
# Load environment variables
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [334]:
# Paths
import os
import sys


# Standard libraries
import pandas as pd
import numpy as np

#ML
#Preprocess
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer

#Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
#Training
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
#Metrics
from sklearn.metrics import  root_mean_squared_error, mean_absolute_error, r2_score, mean_squared_error

import shap
import joblib

#from sklearn.impute import SimpleImputer, KNNImputer
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.naive_bayes import GaussianNB



In [335]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [336]:
fires_dt.head()

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


# Get X and Y

Create the features data frame and target data.

In [337]:
#Scaled variables
numeric_variable = ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'coord_x','coord_y']

#One-hot encoding variables
categorical_variable = ['month', 'day']

#Non-Sacalable variables; Coordinates should not be scaled for this project
#numeric_ns_variable = ['coord_x', 'coord_y']



In [338]:
#All features = X (predictors)
feature_df = fires_dt.drop(columns="area")

#Target data = y (response)
target_df = fires_dt['area']

#Numeric scalable features
#numeric_feature_df = fires_dt[numeric_variable]

#Numeric no scalable features
#numeric_ns_feature_df = fires_dt[numeric_ns_variable]

#Categorical fetures
#categorical_feature_df = fires_dt[categorical_variable]

#X
X= feature_df

#Y
Y = target_df
#Log-Transform target to handle skewness
Y = np.log1p(Y)


In [339]:


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [340]:
#Column Transform

column_transform_01 = ColumnTransformer(
    transformers=[
    ("numeric_scalable", StandardScaler(), numeric_variable),
    ("categorical", OneHotEncoder(handle_unknown="infrequent_if_exist",drop='first'),categorical_variable)
], remainder= "passthrough"

)
preproc1 = column_transform_01

preproc1

ColumnTransformer(remainder='passthrough',
                  transformers=[('numeric_scalable', StandardScaler(),
                                 ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh',
                                  'wind', 'rain', 'coord_x', 'coord_y']),
                                ('categorical',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [341]:
pipe_pr2 = Pipeline([
    ('standardizer', StandardScaler()),
    ('transform', PowerTransformer())
])


ctramsform = ColumnTransformer(
    [
    ('numeric_yj', pipe_pr2, numeric_variable),
    ('categorical', OneHotEncoder(handle_unknown="infrequent_if_exist", drop='first'), categorical_variable)
    ], 
    remainder='passthrough'
)

preproc2 = ctramsform
preproc2

ColumnTransformer(remainder='passthrough',
                  transformers=[('numeric_yj',
                                 Pipeline(steps=[('standardizer',
                                                  StandardScaler()),
                                                 ('transform',
                                                  PowerTransformer())]),
                                 ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh',
                                  'wind', 'rain', 'coord_x', 'coord_y']),
                                ('categorical',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [342]:
#Regressors:
#Baseline: KNN-Regressor
#Advanced model: RandomForestRegressor

# Pipeline A = preproc1 + baseline
# Pipeline B = preproc2 + baseline
# Pipeline C = preproc1 + advanced model
# Pipeline D = preproc2 + advanced model

In [343]:
baseline_model = KNeighborsRegressor()
advanced_model = RandomForestRegressor(random_state=42)

In [344]:
pipelines = {
    'pipeline_a': Pipeline([('preprocessing', preproc1), ('regressor', baseline_model)]),
    'pipeline_b': Pipeline([('preprocessing', preproc2), ('regressor', baseline_model)]),
    'pipeline_c': Pipeline([('preprocessing', preproc1), ('regressor', advanced_model)]),
    'pipeline_d': Pipeline([('preprocessing', preproc2), ('regressor', advanced_model)])
}



# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [345]:
scoring = {
    'RMSE': 'neg_root_mean_squared_error',
    'MAE': 'neg_mean_absolute_error',
    'R2': 'r2'
}

In [346]:
#param_grid knn
param_grid_knn = {
    'regressor_1__n_neighbors': range(1, 200),  # Odd values to prevent ties
    'regressor_1__weights': ['uniform', 'distance'],  # Uniform (equal weight), Distance (inverse distance weighted)
    'regressor_1__metric': ['euclidean', 'manhattan', 'minkowski']  # Distance metrics
}



In [347]:
#param_grid random forest

param_grid_rf = {
    'regressor_2__n_estimators': [50, 100, 200, 300],  # Number of trees
    'regressor_2__max_depth': [None, 5, 10, 20],  # Tree depth
    'regressor_2__min_samples_split': [2, 5, 10],  # Minimum samples to split a node
    'regressor_2__min_samples_leaf': [1, 2, 4],  # Minimum samples per leaf
    'regressor_2__bootstrap': [True, False]  # Whether to sample with replacement
}


In [348]:
#knn-r paramaters
#1. n_neighbours
r1_n_neighbors = range(1,200)

#random_forest paramaters
#1. estimator
r2_estimator = [50, 100, 200, 300]

#param_grids
param_grids = {
    'pipeline_a': {'regressor__n_neighbors': r1_n_neighbors},
    'pipeline_b': {'regressor__n_neighbors': r1_n_neighbors},
    'pipeline_c': {'regressor__n_estimators': r2_estimator},
    'pipeline_d': {'regressor__n_estimators': r2_estimator}
}

In [349]:
best_models = {}
for name, pipeline in pipelines.items():
    grid_cv = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grids[name],
        cv = 5,
        scoring=scoring,
        refit='RMSE',
        n_jobs=-1  # Use all CPU cores
        #,verbose=2  # Show progress
    )
    grid_cv.fit(X_train, y_train)
    best_models[name] = grid_cv.best_estimator_
    print(f"{name} Best Params: {grid_cv.best_params_}")
    print(f"{name} Best Score (RMSE): {-grid_cv.best_score_}")

pipeline_a Best Params: {'regressor__n_neighbors': 58}
pipeline_a Best Score (RMSE): 1.3681680374170728
pipeline_b Best Params: {'regressor__n_neighbors': 179}
pipeline_b Best Score (RMSE): 1.371640997919673
pipeline_c Best Params: {'regressor__n_estimators': 300}
pipeline_c Best Score (RMSE): 1.4602435311776851
pipeline_d Best Params: {'regressor__n_estimators': 300}
pipeline_d Best Score (RMSE): 1.4600993544161176


In [350]:
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)**0.5
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name} | RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")

pipeline_a | RMSE: 1.5017, MAE: 1.2203, R2: -0.0260
pipeline_b | RMSE: 1.4824, MAE: 1.2076, R2: 0.0001
pipeline_c | RMSE: 1.5195, MAE: 1.2007, R2: -0.0505
pipeline_d | RMSE: 1.5178, MAE: 1.1997, R2: -0.0481


# Evaluate

+ Which model has the best performance?

In [356]:
best_model = best_models['pipeline_c']

# Export

+ Save the best performing model to a pickle file.

In [359]:
joblib.dump(best_model, '../../05_src/data/fires//best_forestfires_model.pkl')

['../../05_src/data/fires//best_forestfires_model.pkl']

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [358]:
# Explain Best Model with SHAP
explainer = shap.Explainer(best_model['regressor'], best_model['preprocessing'].transform(X_train))
shap_values = explainer(best_model['preprocessing'].transform(X_test))



ExplainerError: Additivity check failed in TreeExplainer! Please ensure the data matrix you passed to the explainer is the same shape that the model was trained on. If your data shape is correct then please report this on GitHub. This check failed because for one of the samples the sum of the SHAP values was 1.398899, while the model output was 1.463733. If this difference is acceptable you can set check_additivity=False to disable this check.

In [ ]:
# Global Explanation
shap.summary_plot(shap_values, best_model['preprocessor'].transform(X_test), feature_names=numeric_variable + list(best_model['preprocessor'].named_transformers_['categorical'].get_feature_names_out()))

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.